In [1]:
import keras.layers as KL
from keras.models import Model
import keras.backend as K
import tensorflow as tf
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
import numpy as np
import os
%matplotlib inline

Using TensorFlow backend.


In [2]:
def building_block(filters,block):
    
#     import random
#     bn = random.randint(100,300,)
    #判断block1和2
    if block != 0: #如果不等于0 那么使用 stride=1
        stride = 1
    else:         #如果等于0 采用stride 2 两倍下采样 也就是 如果是 building1 使用2倍下采样
        stride = 2

    def f(x):
        #主通路结构
        y = KL.Conv2D(filters=filters,kernel_size=(1,1),strides=stride)(x)
        y = KL.BatchNormalization(axis=3)(y)
        y = KL.Activation('relu')(y)

        y = KL.Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(y) #注意这里没有stride使用padding same就是保证size相同
        y = KL.BatchNormalization(axis=3)(y)
        y = KL.Activation('relu')(y)

        #主通路输出
        y = KL.Conv2D(filters=4*filters,kernel_size=(1,1))(y)
        y = KL.BatchNormalization(axis=3)(y)

        #判断是哪个block 设定不同的 shortcut支路参数
        if block == 0 : #如果是0 那么就是block1的通路
            shortcut = KL.Conv2D(filters=4*filters,kernel_size=(1,1),strides=stride)(x)
            shortcut = KL.BatchNormalization(axis=3)(shortcut)
        else:
            #如果不等于0 那就是block2  那么就直接接input的tensor
            shortcut = x

        #主通路和shortcut 相加
        y = KL.Add()([y,shortcut]) #y主 shortcut支路 直接通过add层相加
        import random
        y = KL.Activation('relu',name='last'+str(random.randint(100,300)))(y)
        return y
    return f

In [3]:
#resnet 主输入函数
def ResNet_Extractor(inputs):
    x = KL.Conv2D(filters=64,kernel_size=(3,3),padding='same')(inputs)
    x = KL.BatchNormalization(axis=3)(x)
    x = KL.Activation('relu')(x)

    #控制调用网络结构feature map 特征图
    #每个stage要有不同的 b12的数量 ，还有 第一个Block1 输入维度后边要迭代（stage）
    filters = 64
    block = [2,2]
    for i,block_num in enumerate(block):
        for block_id in range(block_num):
            x = building_block(filters=filters,block=block_id)(x)
        filters *= 2 #每个stage double filter个数

    return x

In [4]:
# share map 和 anchor 的提取

In [7]:
def RpnNet(featuremap, k=9):
    # 共享
    shareMap = KL.Conv2D(filters=256,kernel_size=(3,3),padding='same',name='sharemap')(featuremap)
    shareMap = KL.Activation('linear')(shareMap)
    # 计算rpn分类前后景
    rpn_classification = KL.Conv2D(filters=2*k,kernel_size=(1,1))(shareMap)
    rpn_classification = KL.Lambda(lambda x:tf.reshape(x,[tf.shape(x)[0],-1,2]))(rpn_classification)
    rpn_classification = KL.Activation('linear',name='rpn_classification')(rpn_classification)
    
    rpn_probability = KL.Activation('softmax',name='rpn_probability')(rpn_classification)
    # 计算回归修正
    rpn_position = KL.Conv2D(filters=4*k,kernel_size=(1,1))(shareMap)
    rpn_position = KL.Activation('linear')(rpn_position)
    rpn_BoundingBox = KL.Lambda(lambda x:tf.reshape(x,[tf.shape(x)[0],-1,4]),name='rpn_POS')(rpn_position)
    
    return rpn_classification,rpn_probability,rpn_BoundingBox
    

In [9]:
x = KL.Input([100,100,3])
featureMap = ResNet_Extractor(x)
rpn_classification,rpn_probability,rpn_BoundingBox = RpnNet(featureMap,k=9)
model = Model(inputs=[x],outputs=[rpn_classification,rpn_probability,rpn_BoundingBox])
plot_model(model=model,to_file='SharedMap.png',show_shapes=True)